### Exercise 03 : Aggregations
* create a connection to the database using the library sqlite3
* get the schema of the table test
* get only the first 10 rows of the table test to check what the table looks like

In [1]:
import pandas as pd
import sqlite3

connection = sqlite3.connect("../data/checking-logs.sqlite")
pd.io.sql.read_sql("pragma table_info(test)", connection)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [2]:
query = "SELECT * FROM test LIMIT 10"
test = pd.io.sql.read_sql(query, connection)
test

,index,uid,labname,first_commit_ts,first_view_ts
0,0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


* find among all the users the minimum value of the delta between the first commit
of the user and the deadline of the corresponding lab using only one query
  * do this by joining the table with the table deadlines
  * the difference should be displayed in hours
  * do not take the lab ’project1’ into account, it has longer deadlines and will be
  an outlier
  * the value should be stored in the dataframe df_min with the corresponding
  uid

In [3]:
query = """SELECT d.uid, MIN(d.delta) AS min 
        FROM (SELECT uid, CAST((julianday(t.first_commit_ts) - julianday(datetime((d.deadlines), "unixepoch"))) * 24 AS INTEGER) as delta
        FROM test t 
        JOIN deadlines d on t.labname = d.labs
        WHERE NOT t.labname = "project1") as d
        """
df_min = pd.read_sql_query(query, connection)
df_min


,uid,min
0,user_30,-202


* do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [4]:
query = """SELECT d.uid, MAX(d.delta) AS max 
        FROM (SELECT uid, CAST((julianday(t.first_commit_ts) - julianday(datetime((d.deadlines), "unixepoch"))) * 24 AS INTEGER) as delta
        FROM test t 
        JOIN deadlines d on t.labname = d.labs
        WHERE NOT t.labname = "project1") as d
        """
df_max = pd.read_sql_query(query, connection)
df_max


,uid,max
0,user_25,-2


* do the same thing but for the average, using only one query, this time your dataframe
should not include the uid column, and the dataframe name is df_avg

In [5]:
query = """SELECT d.uid, AVG(d.delta) AS max 
        FROM (SELECT uid, CAST((julianday(t.first_commit_ts) - julianday(datetime((d.deadlines), "unixepoch"))) * 24 AS INTEGER) as delta
        FROM test t 
        JOIN deadlines d on t.labname = d.labs
        WHERE NOT t.labname = "project1") as d
        """
df_avg = pd.read_sql_query(query, connection)
df_avg

,uid,max
0,user_1,-89.125


* we want to test the hypothesis that the users who visited the newsfeed just a few
times have the lower delta between the first commit and the deadline. To do this,
you need to calculate the correlation coefficient between the number of pageviews
and the difference
  * using only one query, create a table with the columns: uid, avg_diff, pageviews
  * uid is the uids that exist in the test
  * avg_diff is the average delta between the first commit and the lab deadline per user
  * pageviews is the number of Newsfeed visits per user
  * do not take the lab ’project1’ into account
  * store it to the dataframe views_diff

In [6]:
query = """SELECT cu.uid, AVG(d.delta) AS avg_diff, cu.count AS pageviews
                FROM(SELECT uid, COUNT(*) AS count FROM pageviews GROUP BY uid) AS cu
            JOIN (SELECT uid, CAST((julianday(t.first_commit_ts) - julianday(datetime((d.deadlines), "unixepoch"))) * 24 AS INTEGER) as delta
                FROM test t 
                JOIN deadlines d on t.labname = d.labs
                WHERE NOT t.labname = "project1") as d
            ON cu.uid = d.uid
            GROUP BY cu.uid, cu.count
            """
views_diff = pd.io.sql.read_sql(query, connection)
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


* use the Pandas method corr() to calculate the correlation coefficient between
the number of pageviews and the difference

In [7]:
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


* close the connection

In [8]:
connection.close()